In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from astropy.io import fits
from sklearn.preprocessing import minmax_scale
import umap
import hdbscan
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import fits

d:\Projects\projects_101\open_source\ml4s-tests\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class ALMADataset(Dataset):
    def __init__(self, file_paths, transforms=None):
        self.file_paths = file_paths
        self.transforms = transforms

    def __len__(self):
        return len(self.file_paths)
    
    def __getitem__(self, idx):
        path = self.file_paths[idx]
        with fits.open(path) as image_data:
            data = image_data[0].data
            if data.ndim > 2:
                data = data.flatten()[:360000].reshape(600, 600)
            data = np.nan_to_num(data)
            data = (data - np.min(data)) / (np.ptp(data) + 1e-8)
            data = data.astype(np.float32)
            data = np.expand_dims(data, axis=0)
        if self.transforms:
            data = self.transforms(data)
        return data     

In [3]:
transform = transforms.Compose([
    transforms.RandomRotation(180), 
    transforms.CenterCrop(600), 
    transforms.Resize((600, 600)), 
])